# Multiprocessing with multiprocessing 

> **TODO**: Short motivational summary

## Learning outcomes

**TODO**

* Understanding of the basic differences between multprocessing and multithreading and why multiprocessing is require
* Can parallelise the mapping of a function over a sequence of arguments using the `multiprocessing` module

* MULTIPROCESSING [THREADPOOL](http://lucasb.eyer.be/snips/python-thread-pool.html)?
* [JOBLIB](https://pythonhosted.org/joblib/parallel.html)?


## ShARC setup for this lesson

If you want to run through this lesson on ShARC it is recommended that you [start a JupyterHub session](http://docs.hpc.shef.ac.uk/en/latest/hpc/jupyterhub.html#using-jupyter-on-sharc) with four CPU cores and 1GB of RAM per core.

## Introduction

In the previous lesson, we saw how we could use Python packages to interact with compiled, non-Python, multi-threaded libraries to speed up linear algebra and FFTW operations by distributing
 work between multiple CPU cores.  Distributing work between threads (and cores) required little effort on our part but we were restricted using just the functions provided by numpy+MKL and 
pyFFTW+FFTW.  What if we want to parallelise code we have written ourselves?

The simplest case is parallelising some Python code so that we can execute it on multiple CPU cores within a single machine.  Before we get into some examples that demonstrate this we first 
need to know a little more about the distinction between **threads** and **processes** and about the Python **Global Interpreter Lock (GIL)** .

## Some background: multi-processing, multi-threading and the Python Global Interpretter Lock (GIL)

Say we're written our own multithreaded C program.  

When we execute a program we get a **process**, which is a running instance of the program.  The operating system manages the execution of many processes.  Associated each process is:

* memory containing the program's instructions
* memory containing the program's data (e.g. variables and constants)
* process metadata

One simple way of achieving parallelism is to run multiple related instances of the same program on a single machine: the program is written so that the first process be started includes an instruction to **fork** (spawn) a (*child*) process and subsequently both processes are able to run in parallel (ideally on separate CPU cores) until one/both exits.  Forking is quite an expensive operation in terms of time and memory as the parent process's memory containing its instructions and data is all copied to the child process.  The (single) program includes separate logic per process, which are differentiated at run-time by the process ID.  In over-simplified pseudo-code this looks something like:

```
process_id = fork();
if process_id == some_value: 
    # do something specific to the child process
    thing_one()
else:
    # do something specific to the parent process
    thing_two()
# the parent process waits for the child to exit
wait()   
```

In the above example both thing_one and thing_two can execute at the same time and use their individual copies of the data (which may diverge after the fork).

A second common approach to single-machine parallelism is multithreading.  Parallelism is achieved by a process internally spawning a additional strand of execution known as a thread, rather than forking a separate child process.  Creating threads is much cheaper than creating processes as all threads within a process (including the initial thread) **share memory**: there is no need to create copies of the program's instructions and of the process's data when a new thread is spawned.  Multi-threading is therefore *usually* preferable to multi-processing for efficient parallelism on a single machine.  However, coordinating multiple threads to ensure they don't trip over each other when accessing the same data strucures is hard **TODO: REFERENCE LATER SECTION?**).

Although multi-threading is used a lot when programming in languages such as C or Java, **multi-threading is often only of very limited use in Python code**.  This is because for a Python thread to do computation it must hold a per-Python-process resource known as the **Global Intepreter Lock** (GIL).  Any thread that does not hold the GIL must wait for it to become free then successfully acquire it before it can do any computation.  The effect is that **only one of the threads of a Python process can do computational work at any given time**.  The need for the GIL relates to fundamental design decisions made early in the development of the most popular Python interpretter (cPython); many developers would like to find a way to free multi-threaded Python from the shackles of the GIL but that is [not a trivial problem to solve](https://docs.python.org/3/c-api/init.html#thread-state-and-the-global-interpreter-lock).  Note: if a Python function calls a non-Python function (e.g. a MKL function) then the non-Python function is not aware of the GIL not bound by it so can spawn multiple threads.  This is what happens when we use the MKL library via numpy and the FFTW library via pyFFTW.

So, in summary, if we want to parallelise a single Python program so it can distribute computation between the multiple cores of a single machine we must use **multi-processing**: we must fork off child Python processes, each with their own data and instruction memory segments.

## The multiprocessing module

The easiest way to create additional Python processes on a machine and give them work to do is using the `multiprocessing` module.  `multiprocessing` isn't something you need to install; it comes as part of Python.  Let's import it:

In [1]:
import multiprocessing

We then need to create a 'pool' of worker (child) processes:

In [30]:
pool = multiprocessing.Pool()

By default this creates as many worker processes as there are accessible CPU cores (including virtual cores if hyperthreading is enabled; see previous lesson):   

In [31]:
pool._processes

8

If we'd wanted we could have specified the number of workers by passing an integer argument to `Pool()`.

Now what useful things can we do with the `multiprocessing` module?  A common use is a **parallel map**.

First we need to make sure we know what a non-parallel (serial) map is.  To *map* in programming refers to applying a single function to a series of arguments then collating the results.  Here's a trivial example:

In [32]:
def f(x):
    return x ** 10    

%time list(map(f, range(10)))

CPU times: user 8 µs, sys: 4 µs, total: 12 µs
Wall time: 16.2 µs


[0,
 1,
 1024,
 59049,
 1048576,
 9765625,
 60466176,
 282475249,
 1073741824,
 3486784401]

Here we mapped the function `f` over a sequence of values (0 to 9 inclusive).  The result of the map operation has been converted to a list for ease of viewing in this Notebook

We can parallelise the above by using multiprocessing to:

1. Split the sequence of arguments to be mapped over into a chunk per worker in our pool
2. On each worker, in parallel: map the function over its sub-sequence
3. Collate the results from all workers and stitch them back together to yield a single sequence 

In [33]:
%time pool.map(f, range(24))

CPU times: user 2.4 ms, sys: 0 ns, total: 2.4 ms
Wall time: 1.73 ms


[0,
 1,
 1024,
 59049,
 1048576,
 9765625,
 60466176,
 282475249,
 1073741824,
 3486784401,
 10000000000,
 25937424601,
 61917364224,
 137858491849,
 289254654976,
 576650390625,
 1099511627776,
 2015993900449,
 3570467226624,
 6131066257801,
 10240000000000,
 16679880978201,
 26559922791424,
 41426511213649]

NB `pool.map` must be called in a separate Notebook cell to the one in which the function to be mapped is defined (otherwise horrible things happen).  If you use `multiprocesing` in a Python script/program rather than a Notebook then there are [similar conditions](https://docs.python.org/3/library/multiprocessing.html#programming-guidelines) on where `pool.map` can be called. 

Looking at the run times above, we can see that `pool.map` was *much slower* the (serial) `map` function, as the function `f` was sufficiently trivial that the overheads associated with:

* importing the `f` function in the worker processes,
* sending the chunks of function arguments to each worker and
* sending return values back to the Pool manager

dominated the execution time.  We need for `f` to be far less trivial to get a speedup from this parallelisation.   